
### AutoGen AgentChat!

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

### First concept: the Model

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

In [3]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollamamodel_client = OllamaChatCompletionClient(model="llama3.2")

### Second concept: The Message

In [5]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to Lisbon", source="user")
message

TextMessage(source='user', models_usage=None, metadata={}, content="I'd like to go to Lisbon", type='TextMessage')

### Third concept: The Agent

In [6]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

### Put it all together with on_messages

In [7]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

"Lisbon? Great choice! Just remember, it's all about the three M's: munch, mingle, and make your flight on time! 🍕✈️🎉"

### Database of ticket prices

In [8]:
import os
import sqlite3

# Delete existing database file if it exists
if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn = sqlite3.connect("tickets.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [9]:
# Populate our database
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

# Some cities!
save_city_price("London", 299)
save_city_price("Lisbon", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [10]:
# Method to get price for a city
def get_city_price(city_name: str) -> float | None:
    """ Get the roundtrip ticket price to travel to the city """
    conn = sqlite3.connect("tickets.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [12]:
get_city_price("Lisbon")

399.0

In [14]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True # enables agent to think about the result of the tool it just used, respond based on it more intelligently
)

#Without reflection, it might just return "$399" or something bland like: “The price is $399.”
#With reflect_on_tool_use=True, the LLM continues: ✈️ Buckle up! A roundtrip from Paris to Tokyo will only cost you $399—cheaper than a croissant in Tokyo (almost)! 🥐💸


In [15]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

[FunctionCall(id='call_xB4av4qF2aPIeCA90WQefqXV', arguments='{"city_name":"Lisbon"}', name='get_city_price')]
[FunctionExecutionResult(content='399.0', name='get_city_price', call_id='call_xB4av4qF2aPIeCA90WQefqXV', is_error=False)]


'Lisbon, huh? Great choice! A roundtrip ticket will cost you just $399. Just remember to bring your appetite for pasteis de nata! 🥐✈️'